<a href="https://colab.research.google.com/github/yf591/sd-model-merge-tool/blob/main/04_Merge_Model_Maker_Ver1_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2モデルの階層マージ（Hugging Face用）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import output

# Hugging Face Hub, PyTorch, その他必要なライブラリをインストール
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # PyTorchを使用して深層学習モデルを操作します。CUDAバージョン（例: `cu118`）を指定
!pip install diffusers transformers accelerate # Stable Diffusionを扱うための主要ライブラリです。モデルのロードや画像生成の操作を簡素化
!pip install safetensors # 安全かつ軽量なモデル保存形式（`.safetensors`）をサポート
!pip install huggingface-hub # Hugging Face Hubからモデルをダウンロード・管理
!pip install opencv-python # 生成した画像の前処理や後処理に使用
!pip install numpy # 数値計算ライブラリで、モデルや画像の操作に使う
!pip install matplotlib # 生成された画像の可視化に使う
!pip install tqdm # プログレスバーの表示
!pip install optuna # ハイパーパラメータ最適化

output.clear()

In [ ]:
import os
import torch
from safetensors.torch import load_file, save_file
from transformers import AutoConfig, AutoModel
from huggingface_hub import hf_hub_download
import requests
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display
import hashlib
import time

In [ ]:
from getpass import getpass
from google.colab import userdata

# Hugging Faceで取得したTokenをこちらに貼る(トークンを非表示で入力)
HF_TOKEN = getpass("Hugging FaceのRead権限のあるHF Tokenを入力してください: ")

In [ ]:
#@title ### 関数の定義（設定）

# ヘルパー関数
def download_model(repo_id, filename, token):
    """Hugging Face Hubからモデルをダウンロード"""
    return hf_hub_download(repo_id=repo_id, filename=filename, token=token)

def load_model(path, device):
    """ファイルパスからモデルをロードする"""
    try:
        if path.startswith("http"):
            if "huggingface.co" in path:
                print(f"HuggingFaceからモデルをロード: {path}")
                repo_id_and_file = path.split("huggingface.co/")[1]
                repo_id = repo_id_and_file.split("/resolve/")[0]
                filename = repo_id_and_file.split("/")[-1]
                path = download_model(repo_id, filename, HF_TOKEN)
                return load_file(path, device=device)
            else:
                print("Error: HTTP URL not recognized, use HuggingFace Model.")
                return None
        if path.startswith("/content/drive"):
          # Google Drive のパスの場合
            print(f"Google Driveからモデルをロード: {path}")
            return load_file(path, device=device)
        else:
          print("Error: Incorrect Model Path.")
          return None
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def merge_weights(model1, model2, alpha, layer_weights, use_layer_alpha):
    """指定されたレイヤーで重みをマージする"""
    merged = {}
    skipped_layers = []  # スキップされたレイヤーを記録するリスト
    for key in model1.keys():
        if key in model2:
            if key.startswith("model.diffusion_model.input_blocks."):
               layer_name = key.split("model.diffusion_model.input_blocks.")[1].split(".")[0]
               if layer_name in layer_weights and use_layer_alpha[layer_name]:
                  layer_weight = layer_weights[layer_name]
               else:
                  layer_weight = alpha
            elif key.startswith("model.diffusion_model.middle_block."):
               layer_name = "M00"
               if layer_name in layer_weights and use_layer_alpha[layer_name]:
                  layer_weight = layer_weights[layer_name]
               else:
                  layer_weight = alpha
            elif key.startswith("model.diffusion_model.output_blocks."):
               layer_name = key.split("model.diffusion_model.output_blocks.")[1].split(".")[0]
               if layer_name in layer_weights and use_layer_alpha[layer_name]:
                 layer_weight = layer_weights[layer_name]
               else:
                  layer_weight = alpha
            else:
               layer_weight = alpha
            try:
                merged[key] = layer_weight * model1[key] + (1 - layer_weight) * model2[key]
            except Exception as e:
                print(f"Skipped key: {key}, Error: {e}")
                skipped_layers.append(key)  # スキップされたレイヤーを記録
                merged[key] = model1[key]
        else:
            merged[key] = model1[key]
    if skipped_layers:
       print("Skipped layers:", skipped_layers)  # スキップされたレイヤーのリストを表示
    return merged


def save_merged_model(merged_weights, output_path):
    """マージ済みモデルを保存"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    save_file(merged_weights, output_path)

In [ ]:
#@title ### UI設定

#@markdown ### ◆ マージするモデルを設定

# UI設定
repo1 = "sazyou-roukaku/chilled_remix" #@param {type:"string"}
file1 = "chilled_remix_v2.safetensors" #@param {type:"string"}
repo2 = "SG161222/Realistic_Vision_V2.0" #@param {type:"string"}
file2 = "Realistic_Vision_V2.0-fp16-no-ema-inpainting.safetensors" #@param {type:"string"}

#@markdown ### ◆ アウトプット先を指定
output_file = "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/checkpoints/merged_model_kaisou_sample1.safetensors" #@param {type:"string"}

In [ ]:
#@title ### マージ比率の設定（Alpha, レイヤーごとのマージ比率）

#@markdown ### ◆ マージ比率Alphaの設定
alpha_slider = widgets.FloatSlider(value=0.75, min=0.0, max=1.0, step=0.001, description='Alpha')
alpha_text = widgets.FloatText(value=0.75, description='')
widgets.jslink((alpha_slider, 'value'), (alpha_text, 'value'))
display(widgets.HBox([alpha_slider, alpha_text]))

#@markdown ### ◆ レイヤーごとのマージ比率の設定
#@markdown ---
#@markdown #### input_blocks

# IN00
layer_weight_IN00_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN00')
layer_weight_IN00_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN00 = widgets.Checkbox(value=False, description='Use IN00 Layer Alpha')
widgets.jslink((layer_weight_IN00_slider, 'value'), (layer_weight_IN00_text, 'value'))
display(widgets.HBox([layer_weight_IN00_slider, layer_weight_IN00_text, use_layer_alpha_IN00]))

# IN01
layer_weight_IN01_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN01')
layer_weight_IN01_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN01 = widgets.Checkbox(value=False, description='Use IN01 Layer Alpha')
widgets.jslink((layer_weight_IN01_slider, 'value'), (layer_weight_IN01_text, 'value'))
display(widgets.HBox([layer_weight_IN01_slider, layer_weight_IN01_text, use_layer_alpha_IN01]))

# IN02
layer_weight_IN02_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN02')
layer_weight_IN02_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN02 = widgets.Checkbox(value=False, description='Use IN02 Layer Alpha')
widgets.jslink((layer_weight_IN02_slider, 'value'), (layer_weight_IN02_text, 'value'))
display(widgets.HBox([layer_weight_IN02_slider, layer_weight_IN02_text, use_layer_alpha_IN02]))

# IN03
layer_weight_IN03_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN03')
layer_weight_IN03_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN03 = widgets.Checkbox(value=False, description='Use IN03 Layer Alpha')
widgets.jslink((layer_weight_IN03_slider, 'value'), (layer_weight_IN03_text, 'value'))
display(widgets.HBox([layer_weight_IN03_slider, layer_weight_IN03_text, use_layer_alpha_IN03]))

# IN04
layer_weight_IN04_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN04')
layer_weight_IN04_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN04 = widgets.Checkbox(value=False, description='Use IN04 Layer Alpha')
widgets.jslink((layer_weight_IN04_slider, 'value'), (layer_weight_IN04_text, 'value'))
display(widgets.HBox([layer_weight_IN04_slider, layer_weight_IN04_text, use_layer_alpha_IN04]))

# IN05
layer_weight_IN05_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN05')
layer_weight_IN05_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN05 = widgets.Checkbox(value=False, description='Use IN05 Layer Alpha')
widgets.jslink((layer_weight_IN05_slider, 'value'), (layer_weight_IN05_text, 'value'))
display(widgets.HBox([layer_weight_IN05_slider, layer_weight_IN05_text, use_layer_alpha_IN05]))

# IN06
layer_weight_IN06_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN06')
layer_weight_IN06_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN06 = widgets.Checkbox(value=False, description='Use IN06 Layer Alpha')
widgets.jslink((layer_weight_IN06_slider, 'value'), (layer_weight_IN06_text, 'value'))
display(widgets.HBox([layer_weight_IN06_slider, layer_weight_IN06_text, use_layer_alpha_IN06]))

# IN07
layer_weight_IN07_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN07')
layer_weight_IN07_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN07 = widgets.Checkbox(value=False, description='Use IN07 Layer Alpha')
widgets.jslink((layer_weight_IN07_slider, 'value'), (layer_weight_IN07_text, 'value'))
display(widgets.HBox([layer_weight_IN07_slider, layer_weight_IN07_text, use_layer_alpha_IN07]))

# IN08
layer_weight_IN08_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN08')
layer_weight_IN08_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN08 = widgets.Checkbox(value=False, description='Use IN08 Layer Alpha')
widgets.jslink((layer_weight_IN08_slider, 'value'), (layer_weight_IN08_text, 'value'))
display(widgets.HBox([layer_weight_IN08_slider, layer_weight_IN08_text, use_layer_alpha_IN08]))

# IN09
layer_weight_IN09_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN09')
layer_weight_IN09_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN09 = widgets.Checkbox(value=False, description='Use IN09 Layer Alpha')
widgets.jslink((layer_weight_IN09_slider, 'value'), (layer_weight_IN09_text, 'value'))
display(widgets.HBox([layer_weight_IN09_slider, layer_weight_IN09_text, use_layer_alpha_IN09]))

# IN10
layer_weight_IN10_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN10')
layer_weight_IN10_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN10 = widgets.Checkbox(value=False, description='Use IN10 Layer Alpha')
widgets.jslink((layer_weight_IN10_slider, 'value'), (layer_weight_IN10_text, 'value'))
display(widgets.HBox([layer_weight_IN10_slider, layer_weight_IN10_text, use_layer_alpha_IN10]))

# IN11
layer_weight_IN11_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='IN11')
layer_weight_IN11_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_IN11 = widgets.Checkbox(value=False, description='Use IN11 Layer Alpha')
widgets.jslink((layer_weight_IN11_slider, 'value'), (layer_weight_IN11_text, 'value'))
display(widgets.HBox([layer_weight_IN11_slider, layer_weight_IN11_text, use_layer_alpha_IN11]))

#@markdown ---
#@markdown #### middle_block
# M00
layer_weight_M00_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='M00')
layer_weight_M00_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_M00 = widgets.Checkbox(value=False, description='Use M00 Layer Alpha')
widgets.jslink((layer_weight_M00_slider, 'value'), (layer_weight_M00_text, 'value'))
display(widgets.HBox([layer_weight_M00_slider, layer_weight_M00_text, use_layer_alpha_M00]))

#@markdown ---
#@markdown #### output_blocks
# OUT00
layer_weight_OUT00_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT00')
layer_weight_OUT00_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT00 = widgets.Checkbox(value=False, description='Use OUT00 Layer Alpha')
widgets.jslink((layer_weight_OUT00_slider, 'value'), (layer_weight_OUT00_text, 'value'))
display(widgets.HBox([layer_weight_OUT00_slider, layer_weight_OUT00_text, use_layer_alpha_OUT00]))

# OUT01
layer_weight_OUT01_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT01')
layer_weight_OUT01_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT01 = widgets.Checkbox(value=False, description='Use OUT01 Layer Alpha')
widgets.jslink((layer_weight_OUT01_slider, 'value'), (layer_weight_OUT01_text, 'value'))
display(widgets.HBox([layer_weight_OUT01_slider, layer_weight_OUT01_text, use_layer_alpha_OUT01]))

# OUT02
layer_weight_OUT02_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT02')
layer_weight_OUT02_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT02 = widgets.Checkbox(value=False, description='Use OUT02 Layer Alpha')
widgets.jslink((layer_weight_OUT02_slider, 'value'), (layer_weight_OUT02_text, 'value'))
display(widgets.HBox([layer_weight_OUT02_slider, layer_weight_OUT02_text, use_layer_alpha_OUT02]))

# OUT03
layer_weight_OUT03_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT03')
layer_weight_OUT03_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT03 = widgets.Checkbox(value=False, description='Use OUT03 Layer Alpha')
widgets.jslink((layer_weight_OUT03_slider, 'value'), (layer_weight_OUT03_text, 'value'))
display(widgets.HBox([layer_weight_OUT03_slider, layer_weight_OUT03_text, use_layer_alpha_OUT03]))

# OUT04
layer_weight_OUT04_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT04')
layer_weight_OUT04_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT04 = widgets.Checkbox(value=False, description='Use OUT04 Layer Alpha')
widgets.jslink((layer_weight_OUT04_slider, 'value'), (layer_weight_OUT04_text, 'value'))
display(widgets.HBox([layer_weight_OUT04_slider, layer_weight_OUT04_text, use_layer_alpha_OUT04]))

# OUT05
layer_weight_OUT05_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT05')
layer_weight_OUT05_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT05 = widgets.Checkbox(value=False, description='Use OUT05 Layer Alpha')
widgets.jslink((layer_weight_OUT05_slider, 'value'), (layer_weight_OUT05_text, 'value'))
display(widgets.HBox([layer_weight_OUT05_slider, layer_weight_OUT05_text, use_layer_alpha_OUT05]))

# OUT06
layer_weight_OUT06_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT06')
layer_weight_OUT06_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT06 = widgets.Checkbox(value=False, description='Use OUT06 Layer Alpha')
widgets.jslink((layer_weight_OUT06_slider, 'value'), (layer_weight_OUT06_text, 'value'))
display(widgets.HBox([layer_weight_OUT06_slider, layer_weight_OUT06_text, use_layer_alpha_OUT06]))

# OUT07
layer_weight_OUT07_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT07')
layer_weight_OUT07_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT07 = widgets.Checkbox(value=False, description='Use OUT07 Layer Alpha')
widgets.jslink((layer_weight_OUT07_slider, 'value'), (layer_weight_OUT07_text, 'value'))
display(widgets.HBox([layer_weight_OUT07_slider, layer_weight_OUT07_text, use_layer_alpha_OUT07]))

# OUT08
layer_weight_OUT08_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT08')
layer_weight_OUT08_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT08 = widgets.Checkbox(value=False, description='Use OUT08 Layer Alpha')
widgets.jslink((layer_weight_OUT08_slider, 'value'), (layer_weight_OUT08_text, 'value'))
display(widgets.HBox([layer_weight_OUT08_slider, layer_weight_OUT08_text, use_layer_alpha_OUT08]))

# OUT09
layer_weight_OUT09_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT09')
layer_weight_OUT09_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT09 = widgets.Checkbox(value=False, description='Use OUT09 Layer Alpha')
widgets.jslink((layer_weight_OUT09_slider, 'value'), (layer_weight_OUT09_text, 'value'))
display(widgets.HBox([layer_weight_OUT09_slider, layer_weight_OUT09_text, use_layer_alpha_OUT09]))

# OUT10
layer_weight_OUT10_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT10')
layer_weight_OUT10_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT10 = widgets.Checkbox(value=False, description='Use OUT10 Layer Alpha')
widgets.jslink((layer_weight_OUT10_slider, 'value'), (layer_weight_OUT10_text, 'value'))
display(widgets.HBox([layer_weight_OUT10_slider, layer_weight_OUT10_text, use_layer_alpha_OUT10]))

# OUT11
layer_weight_OUT11_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.001, description='OUT11')
layer_weight_OUT11_text = widgets.FloatText(value=0.5, description='')
use_layer_alpha_OUT11 = widgets.Checkbox(value=False, description='Use OUT11 Layer Alpha')
widgets.jslink((layer_weight_OUT11_slider, 'value'), (layer_weight_OUT11_text, 'value'))
display(widgets.HBox([layer_weight_OUT11_slider, layer_weight_OUT11_text, use_layer_alpha_OUT11]))



# レイヤーごとの比率設定
layer_weights = {
    "IN00": layer_weight_IN00_slider,
    "IN01": layer_weight_IN01_slider,
    "IN02": layer_weight_IN02_slider,
    "IN03": layer_weight_IN03_slider,
    "IN04": layer_weight_IN04_slider,
    "IN05": layer_weight_IN05_slider,
    "IN06": layer_weight_IN06_slider,
    "IN07": layer_weight_IN07_slider,
    "IN08": layer_weight_IN08_slider,
    "IN09": layer_weight_IN09_slider,
    "IN10": layer_weight_IN10_slider,
    "IN11": layer_weight_IN11_slider,
    "M00": layer_weight_M00_slider,
    "OUT00": layer_weight_OUT00_slider,
    "OUT01": layer_weight_OUT01_slider,
    "OUT02": layer_weight_OUT02_slider,
    "OUT03": layer_weight_OUT03_slider,
    "OUT04": layer_weight_OUT04_slider,
    "OUT05": layer_weight_OUT05_slider,
    "OUT06": layer_weight_OUT06_slider,
    "OUT07": layer_weight_OUT07_slider,
    "OUT08": layer_weight_OUT08_slider,
    "OUT09": layer_weight_OUT09_slider,
    "OUT10": layer_weight_OUT10_slider,
    "OUT11": layer_weight_OUT11_slider
}

# 各レイヤーをマージ比率Alphaを適用するか否か（チェックボックス）
use_layer_alpha = {
    "IN00": use_layer_alpha_IN00,
    "IN01": use_layer_alpha_IN01,
    "IN02": use_layer_alpha_IN02,
    "IN03": use_layer_alpha_IN03,
    "IN04": use_layer_alpha_IN04,
    "IN05": use_layer_alpha_IN05,
    "IN06": use_layer_alpha_IN06,
    "IN07": use_layer_alpha_IN07,
    "IN08": use_layer_alpha_IN08,
    "IN09": use_layer_alpha_IN09,
    "IN10": use_layer_alpha_IN10,
    "IN11": use_layer_alpha_IN11,
     "M00": use_layer_alpha_M00,
    "OUT00": use_layer_alpha_OUT00,
    "OUT01": use_layer_alpha_OUT01,
    "OUT02": use_layer_alpha_OUT02,
    "OUT03": use_layer_alpha_OUT03,
    "OUT04": use_layer_alpha_OUT04,
    "OUT05": use_layer_alpha_OUT05,
    "OUT06": use_layer_alpha_OUT06,
    "OUT07": use_layer_alpha_OUT07,
    "OUT08": use_layer_alpha_OUT08,
    "OUT09": use_layer_alpha_OUT09,
    "OUT10": use_layer_alpha_OUT10,
    "OUT11": use_layer_alpha_OUT11
}


#@markdown ---
#@markdown ### **チェックボックスについて**
#@markdown #### - チェックボックスに **✓ を入れる (オン, `True`) と、そのレイヤーは `alpha` 値でマージ** されます。
#@markdown #### - チェックボックスに **✓ を入れない (オフ, `False`) と、そのレイヤーは個別に設定された比率でマージ** されます。

In [ ]:
#@title ###実行
try:
    print("モデルのダウンロードを開始します...")
    path1 = download_model(repo1, file1, HF_TOKEN)
    path2 = download_model(repo2, file2, HF_TOKEN)
    alpha = alpha_slider.value # sliderの値をalphaにセット
    print("モデルをロード中...")
    model1 = load_file(path1, device="cuda" if torch.cuda.is_available() else "cpu")
    model2 = load_file(path2, device="cuda" if torch.cuda.is_available() else "cpu")

    # ここにモデルのキーを出力するコードを追加
    print("model1 keys:", list(model1.keys())[:5] , "...")
    print("model2 keys:", list(model2.keys())[:5] , "...")

     # レイヤーごとの比率をsliderから取得
    for layer_name in layer_weights:
        layer_weights[layer_name] = layer_weights[layer_name].value

    print("モデルをレイヤーごとにマージ中...")
    merged_weights = merge_weights(model1, model2, alpha, layer_weights, use_layer_alpha)

    print(f"マージされたモデルを保存します: {output_file}")
    save_merged_model(merged_weights, output_file)
    print("マージ完了！")

except Exception as e:
    print(f"エラーが発生しました: {e}")